In [ ]:
import plotly.express as px
from rindti.models import NoisyNodesClassModel, ClassificationModel, InfoGraphModel, GraphLogModel, PfamModel, BGRLModel
from rindti.models.bgrl import Encoder
from rindti.utils.data import Dataset
import pickle
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE

In [ ]:
with open('results/prepare_all/random_posneg_under_classification_template_label_none.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
prots = pd.read_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/merged_500.pkl")

In [ ]:
vc = prots['fam'].value_counts()
prots_sub = prots[prots['fam'].isin(vc.head(10).index)]

In [ ]:
prots_sub

In [ ]:
class TestModel(ClassificationModel):
    def embed_prot(self, x, edge_index, **kwargs):
        x = self.prot_feat_embed(x)
        x = self.prot_node_embed(x, edge_index)
        return self.prot_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)
    
    def embed_drug(self, x, edge_index, **kwargs):
        x = self.drug_feat_embed(x)
        x = self.drug_node_embed(x, edge_index)
        return self.drug_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)

In [ ]:
def embed_prot(encoder, x, edge_index, **kwargs):
    embed, _ = encoder(x, edge_index, batch=None)
    return embed.detach().numpy().reshape(-1)

In [ ]:

model = BGRLModel.load_from_checkpoint('tb_logs/bgrl/version_3/checkpoints/epoch=210-step=67097.ckpt')
model.eval()
encoder = model.student_encoder

In [ ]:
embeds = {}
prots = data['prots']
for id, prot in prots_sub['data'].to_dict().items():
    emb = embed_prot(encoder, **prot)
    embeds[id] = emb
prot_embeds = pd.DataFrame(embeds).T

In [ ]:
tsne = TSNE()
x = tsne.fit_transform(prot_embeds)


In [ ]:
x = pd.DataFrame(index=prots_sub.index, data=x).join(prots_sub)

In [ ]:
px.scatter(x, 0, 1, hover_name=x.index, hover_data=['fam'], color='fam')

In [ ]:
sns.clustermap(prot_embeds)

In [ ]:
help(sns.clustermap)